In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas, sys, cv2, os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import SGD
from keras.utils import plot_model
import keras, h5py
from Helpers.SteelDataGenerator import SteelDataGenerator as sd_gen
from keras.callbacks import TensorBoard

C:\Programs\Anaconda3\envs\ai\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
neu_data_dir = 'neu-data'
train_dir = os.path.join(neu_data_dir, 'train')
val_dir = os.path.join(neu_data_dir, 'val')
test_dir = os.path.join(neu_data_dir, 'test')
img_width, img_height = 200, 200
input_shape = (img_width, img_height, 3)
epochs = 1000
batch_size = 8
nb_train_samples = 1254
nb_validation_samples = 270    
nb_test_samples = 276
classes = 6
train_generator, val_generator, test_generator = sd_gen.get_data_generators(train_dir, val_dir, test_dir, img_width, img_height, batch_size)

Found 1254 images belonging to 6 classes.
Found 270 images belonging to 6 classes.
Found 276 images belonging to 6 classes.


In [3]:
check_point = keras.callbacks.ModelCheckpoint('LeNet5', monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early_stopping = keras.callbacks.EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')
tb_callback = TensorBoard(log_dir='Graph',
                         histogram_freq=0,
                         write_graph=True,
                         write_images=True)

In [4]:
# btch = train_generator.next()[0][0]
# plt.imshow(btch)
# plt.show()

In [5]:

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(6))
model.add(Activation('softmax'))


In [6]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 198, 198, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 198, 198, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 99, 99, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 97, 97, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 97, 97, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 46, 46, 64)        18496     
__________

In [8]:
model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=val_generator,
        validation_steps=nb_validation_samples // batch_size,
        verbose=1, 
        callbacks=[check_point, early_stopping, tb_callback])

Epoch 1/1000
156/156 [==============================] - 12s 74ms/step - loss: 1.6146 - acc: 0.3002 - val_loss: 1.1738 - val_acc: 0.5530

Epoch 00001: val_acc improved from -inf to 0.55303, saving model to LeNet5
Epoch 2/1000
156/156 [==============================] - 11s 72ms/step - loss: 1.1835 - acc: 0.4936 - val_loss: 1.0081 - val_acc: 0.5379

Epoch 00002: val_acc did not improve
Epoch 3/1000
156/156 [==============================] - 11s 72ms/step - loss: 1.0222 - acc: 0.5641 - val_loss: 0.7926 - val_acc: 0.6742

Epoch 00003: val_acc improved from 0.55303 to 0.67424, saving model to LeNet5
Epoch 4/1000
156/156 [==============================] - 11s 73ms/step - loss: 0.9744 - acc: 0.6135 - val_loss: 0.6868 - val_acc: 0.7121

Epoch 00004: val_acc improved from 0.67424 to 0.71212, saving model to LeNet5
Epoch 5/1000
156/156 [==============================] - 11s 72ms/step - loss: 0.8643 - acc: 0.6522 - val_loss: 0.6389 - val_acc: 0.7348

Epoch 00005: val_acc improved from 0.71212 to 0

In [67]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
print("Точность на тестовых данных: %.2f%%" % (scores[1]*100))

Точность на тестовых данных: 15.81%


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 198, 198, 32)      896       
_________________________________________________________________
activation_6 (Activation)    (None, 198, 198, 32)      0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 99, 99, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 97, 97, 32)        9248      
_________________________________________________________________
activation_7 (Activation)    (None, 97, 97, 32)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 46, 46, 64)        18496     
__________